In [41]:
from sklearn.naive_bayes import CategoricalNB
import json
import pandas as pd


In [42]:
with open('masterchef_data_cleaned.json', 'r') as file:
    data = json.load(file)

In [43]:
data.keys()

dict_keys(['United States', 'Canada', 'Brazil'])

In [44]:
# data.keys()
US = pd.DataFrame(data['United States'])
Canada = pd.DataFrame(data['Canada'])
Brazil = pd.DataFrame(data['Brazil'])


In [45]:
df = pd.concat([US, Canada, Brazil])

In [46]:
def winner(row):
    return row['evaluations'][-1] == 'WINNER'



In [47]:
pd.DataFrame(df[(df['season'] == 1) & (df['nation'] == 'United States')]['evaluations'])
new_df = df.copy()
new_df

,season,name,age,home,occupation,evaluations,nation
0,1,Whitney Miller,22,"Poplarville, Mississippi",College Student,"[WIN_I, IN_I, IN_I, HIGH_I, IN_I, IN_I, IN_I, ...",United States
1,1,David Miller,29,"Boston, Massachusetts",Software Engineer,"[IN_I, IN_I, IN_I, IN_I, IN_I, WIN_T, IN_I, IN...",United States
2,1,Lee Knaz,27,"Venice, California",Bartender,"[IN_I, IN_I, WIN_T, IN_I, IN_I, IN_I, WIN_I, W...",United States
3,1,Sheetal Bhagat,37,"Chicago, Illinois",Teacher,"[IN_I, IN_I, WIN_T, IN_I, IN_I, WIN_T, HIGH_I,...",United States
4,1,Sharone Hakman,28,"Los Angeles, California",Financial Advisor,"[IN_I, IN_I, IN_I, WIN_I, IMM, WIN_T, HIGH_I, ...",United States
...,...,...,...,...,...,...,...
168,10,Camila Barreto,31,Duque de Caxias,Model,"[LOW_T, IN_I, IN_I, IMM, LOW_I, LOW_T, LOW_I, ...",Brazil
169,10,Mariane Souza,29,Serrana,Bank clerk,"[IN_I, IMM, IN_I, HIGH_I, WIN_T, WIN_I, IMM, E...",Brazil
170,10,Diego Lima,25,Rio de Janeiro,Navy military,"[HIGH_I, IMM, HIGH_I, IMM, IMM, IN_I, ELIM]",Brazil
171,10,Eduardo Riba,31,Joinville,Electrician,"[IN_I, IMM, IN_I, IN_I, ELIM]",Brazil


In [48]:
new_df['won'] = new_df.apply(winner,axis=1)

In [49]:
winners = new_df[new_df['won']]

# Function to generate season ID
def generate_season_id(row):
    country_id = {"USA": 1, "Canada": 2, "Brazil": 3}
    
    season_id = row['season'] * 10 + country_id.get(row['nation'], 0)
    
    return season_id




new_df['seasonID'] = new_df.apply(generate_season_id, axis=1)



In [50]:
new_df[(new_df['nation'] == "Brazil") & (new_df['season'].isin([3, 7, 9, 10]))]['evaluations'].loc[139]

['IN_I',
 'IMM',
 'WIN_I',
 'IMM',
 'WIN_I',
 'IMM',
 'IN_I',
 '00bffd-HIGH',
 'WIN_T',
 'WIN_I',
 'IMM',
 'LOW_T',
 '00bffd-HIGH',
 'WIN_I',
 'IMM',
 'WIN_I',
 'IN_I',
 'IN_I',
 'LOW_I',
 'WIN_I',
 'IMM',
 'IN_I',
 'IN_I',
 'IN_I',
 'WIN_I',
 'WIN_I',
 'IMM',
 '#ffd700-WINNER']

In [51]:
winners

,season,name,age,home,occupation,evaluations,nation,won
0,1,Whitney Miller,22,"Poplarville, Mississippi",College Student,"[WIN_I, IN_I, IN_I, HIGH_I, IN_I, IN_I, IN_I, ...",United States,True
14,2,Jennifer Behm,34,"Wilmington, Delaware",Realtor,"[IN_I, IN_I, LOW_I, LOW_I, IN_I, WIN_T, WIN_I,...",United States,True
32,3,Christine Hà,32,"Houston, Texas",MFA Student,"[IN_I, IN_I, IN_I, HIGH_I, WIN_I, WIN_T, IN_I,...",United States,True
50,4,Luca Manfè,31,"Astoria, New York",Restaurant Manager,"[IN_I, IN_I, WIN_T, WIN_I, IMM, LOW_I, IN_I, I...",United States,True
69,5,Courtney Lapresi,25,"Philadelphia, Pennsylvania",Aerial Dancer,"[WIN_I, IMM, IN_I, IN_I, LOW_I, WIN_T, IN_I, W...",United States,True
91,6,Claudia Sandoval,31,"La Mesa, California",Events Manager,"[IN_I, IMM_2, IMM, IN_I, IN_I, IN_I, IN_I, IN_...",United States,True
113,7,Shaun O'Neale,33,"Las Vegas, Nevada",DJ,"[IN_I, IN_I, IN_T, IN_I, IN_I, IN_I, IMM, WIN_...",United States,True
133,8,Dino Angelo Luciano,28,"Bensonhurst, New York",Dancer,"[IN_I, IMM, IN_I, IN_I, LOW_I, IN_I, IN_I, IMM...",United States,True
153,9,Gerron Hurt,25,"Louisville, Kentucky",English Teacher,"[IN_I, IMM, LOW_I, LOW_I, WIN_T, IN_I, IMM_2, ...",United States,True
177,10,Dorian Hunter,45,"Cartersville, Georgia",Creeler,"[IN_I, IN_I, LOW_T, IN_I, IMM, IN_I, HIGH_I, L...",United States,True
